# Inference on SLOSH tiles using Pix2PixHD trained on Masks_v1_sloshed 

In [ ]:
#!pip install dominate ; pip install torch==1.5.1+cu101 torchvision==0.6.1+cu101 -f https://download.pytorch.org/whl/torch_stable.html

In [20]:
#Generate predictions over the test set. Only first 150 samples. We do not have SLOSH for Hurricane Matthew
!cd ../.. ; python src/models/Pix2pixHD/test.py --slosh_inference --checkpoints_dir pretrained/Pix2pixHD/  --phase test --how_many 150 --name conditional_binary_sloshed --dataroot xBD --dataset_mode physics_aligned_bin_sloshed --no_instance --label_nc 0 --input_nc 4 --gpu_ids 0 --results_dir temp/Pix2pixHD/


------------ Options -------------
aspect_ratio: 1.0
batchSize: 1
checkpoints_dir: pretrained/Pix2pixHD/
cluster_path: features_clustered_010.npy
data_type: 32
dataroot: xBD
dataset_mode: physics_aligned_bin_sloshed
display_winsize: 512
engine: None
export_onnx: None
feat_num: 3
fineSize: 512
fp16: False
gpu_ids: [0]
how_many: 150
input_nc: 4
instance_feat: False
isTrain: False
label_feat: False
label_nc: 0
loadSize: 1024
load_features: False
local_rank: 0
max_dataset_size: inf
model: pix2pixHD
nThreads: 2
n_blocks_global: 9
n_blocks_local: 3
n_clusters: 10
n_downsample_E: 4
n_downsample_global: 4
n_local_enhancers: 1
name: conditional_binary_sloshed
nef: 16
netG: global
ngf: 64
niter_fix_global: 0
no_flip: False
no_instance: True
norm: instance
ntest: inf
onnx: None
output_nc: 3
phase: test
resize_or_crop: scale_width
results_dir: temp/Pix2pixHD/
serial_batches: False
slosh_inference: True
tf_log: False
use_dropout: False
use_encoded_image: False
verbose: False
which_epoch: latest
---

### We move things around at the moment in ./Temp/
The model output is placed into ./temp/ .We use this directory to work, then we carefully selects what goes into ./results/.
./temp/ does not get uploaded into our bucket, only ./results/ do!

In [21]:
# Move synthesized results to a folder to be used by the segmentation network
!cd ../../temp/Pix2pixHD/conditional_binary_sloshed/test_latest/ ; mkdir to_segment; cp -R images/*synthesized*.jpg to_segment

In [22]:
#Create Segmentation Masks
#Generate masks for all of the data generated by pix2pixHD_baseline
! cd ../.. ; python src/models/Pix2pix-CycleGAN/test.py --phase to_segment --dataroot temp/Pix2pixHD/conditional_binary_sloshed/test_latest/ --direction AtoB --results_dir ./temp/Pix2pixHD/conditional_binary_sloshed_masks --dataset_mode single --model pix2pix --checkpoints_dir ./pretrained/Pix2pix-CycleGAN/flood_segmentation/scratch_1024_plus/ --name . --num_test 2000 --no_flip --gpu_ids 1 --max_dataset_size 2000 --batch_size 1  --load_size 1024 --crop_size 1024


----------------- Options ---------------
             aspect_ratio: 1.0                           
               batch_size: 1                             
          checkpoints_dir: ./pretrained/Pix2pix-CycleGAN/flood_segmentation/scratch_1024_plus/	[default: ./checkpoints]
                crop_size: 1024                          	[default: 256]
                 dataroot: temp/Pix2pixHD/conditional_binary_sloshed/test_latest/	[default: None]
             dataset_mode: single                        	[default: aligned]
                direction: AtoB                          
          display_winsize: 256                           
                    epoch: latest                        
                     eval: False                         
                  gpu_ids: 1                             	[default: 0]
                init_gain: 0.02                          
                init_type: normal                        
                 input_nc: 3                           

### Now we copy our generated images into the right ./Results/. directory

In [26]:
! cd ../.. ; cp -R temp/Pix2pixHD/conditional_binary_sloshed/test_latest/images/* results/Pix2pixHD/conditional_binary_sloshed_inference/ ; cp -R temp/Pix2pixHD/conditional_binary_sloshed_masks/to_segment_latest/images/*fake_B.png results/Pix2pixHD/conditional_binary_sloshed_inference/